In [1]:
!pip install requests
!pip install pymysql
import datetime
import pymysql
import requests
import pandas as pd

In [3]:
API_KEY = "OOte4ONALmXlJRfnIHAOMpXHYeszi5h1zN10Vq9T"

In [13]:
asteroids_data = []
target = 10000

url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-07&api_key={API_KEY}"

while len(asteroids_data) < target:
    response = requests.get(url)
    data = response.json()
    details = data['near_earth_objects']
    for date,ast_details in details.items():
          for ast in ast_details:
                asteroids_data.append(dict(
                id = int(ast['id']), 
                name = ast['name'],
                absolute_magnitude_h = float(ast['absolute_magnitude_h']),
                estimated_diameter_min_km = float(ast['estimated_diameter']['kilometers']['estimated_diameter_min']),
                estimated_diameter_max_km = float(ast['estimated_diameter']['kilometers']['estimated_diameter_max']),
                is_potentially_hazardous_asteroid = bool(ast['is_potentially_hazardous_asteroid']),
                neo_reference_id = int(ast['neo_reference_id']),
                close_approach_date = datetime.datetime.strptime(ast['close_approach_data'][0]['close_approach_date'],'%Y-%m-%d').date(),
                relative_velocity_kmph = float(ast['close_approach_data'][0]['relative_velocity']['kilometers_per_hour']),
                astronomical = float(ast['close_approach_data'][0]['miss_distance']['astronomical']),
                miss_distance_km = float(ast['close_approach_data'][0]['miss_distance']['kilometers']),
                miss_distance_lunar = float(ast['close_approach_data'][0]['miss_distance']['lunar']),
                orbiting_body = ast['close_approach_data'][0]['orbiting_body']

                ))

                if len(asteroids_data) == target:
                    break
          if len(asteroids_data) == target:
              break
    url = data['links']['next']
    


In [21]:
df = pd.DataFrame(asteroids_data)
df['close_approach_date'].unique()

array([datetime.date(2024, 1, 2), datetime.date(2024, 1, 1),
       datetime.date(2024, 1, 4), datetime.date(2024, 1, 3),
       datetime.date(2024, 1, 6), datetime.date(2024, 1, 5),
       datetime.date(2024, 1, 7), datetime.date(2024, 1, 13),
       datetime.date(2024, 1, 12), datetime.date(2024, 1, 8),
       datetime.date(2024, 1, 9), datetime.date(2024, 1, 11),
       datetime.date(2024, 1, 10), datetime.date(2024, 1, 15),
       datetime.date(2024, 1, 14), datetime.date(2024, 1, 17),
       datetime.date(2024, 1, 16), datetime.date(2024, 1, 19),
       datetime.date(2024, 1, 18), datetime.date(2024, 1, 24),
       datetime.date(2024, 1, 23), datetime.date(2024, 1, 25),
       datetime.date(2024, 1, 20), datetime.date(2024, 1, 22),
       datetime.date(2024, 1, 21), datetime.date(2024, 1, 26),
       datetime.date(2024, 1, 28), datetime.date(2024, 1, 27),
       datetime.date(2024, 1, 29), datetime.date(2024, 1, 31),
       datetime.date(2024, 1, 30), datetime.date(2024, 2, 3),
  

In [23]:
print(asteroids_data[9999])

{'id': 54527592, 'name': '(2025 HJ3)', 'absolute_magnitude_h': 26.245, 'estimated_diameter_min_km': 0.0149814889, 'estimated_diameter_max_km': 0.0334996276, 'is_potentially_hazardous_asteroid': False, 'neo_reference_id': 54527592, 'close_approach_date': datetime.date(2025, 4, 8), 'relative_velocity_kmph': 19683.4288240534, 'astronomical': 0.0324580276, 'miss_distance_km': 4855651.793361212, 'miss_distance_lunar': 12.6261727364, 'orbiting_body': 'Earth'}


In [25]:
len(asteroids_data)

10000

In [29]:
myconnection = pymysql.connect(
                host = "127.0.0.1",
                user="root",
                passwd="Aadhu#1207",
                database = "NEO")
cursor_NEO = myconnection.cursor()

cursor_NEO.execute("""create table if not exists asteroids(
                    id int,
                    name varchar(30),
                    absolute_magnitude_h float,
                    estimated_diameter_min_km float,
                    estimated_diameter_max_km float,
                    is_potentially_hazardous_asteroid bool)""")
                    
cursor_NEO.execute("""create table if not exists close_approach(
                    neo_reference_id int,
                    close_approach_date date,
                    relative_velocity_kmph float,
                    astronomical float,
                    miss_distance_km float,
                    miss_distance_lunar float,
                    orbiting_body varchar(10))""")

0

In [31]:
# Inserting data into Table 1 using cursor

insert_query_1 ='''INSERT INTO asteroids(id, name, absolute_magnitude_h,estimated_diameter_min_km,estimated_diameter_max_km,is_potentially_hazardous_asteroid) VALUES (%s,%s,%s,%s,%s,%s)'''

# Inserting data into Table 2 using cursor

insert_query_2 ='''INSERT INTO close_approach(neo_reference_id,close_approach_date,relative_velocity_kmph,astronomical,miss_distance_km,miss_distance_lunar,orbiting_body) VALUES (%s,%s,%s,%s,%s,%s,%s)'''

# Loop through the list - asteroids_data

for i in asteroids_data:
    values = (i["id"], i["name"], i["absolute_magnitude_h"],i["estimated_diameter_min_km"],i["estimated_diameter_max_km"],i["is_potentially_hazardous_asteroid"])
    cursor_NEO.execute(insert_query_1, values)

for i in asteroids_data:
    values = (i["neo_reference_id"],i["close_approach_date"],i["relative_velocity_kmph"],i["astronomical"],i["miss_distance_km"],i["miss_distance_lunar"],i["orbiting_body"])
    cursor_NEO.execute(insert_query_2, values)    

# Commit changes and close connection
myconnection.commit()
myconnection.close()